# Code to convert the excel file to a usable json

In [32]:
from flask import Flask, render_template, jsonify
from flask_pymongo import PyMongo
import json
import ev_ranges

def ranges():
    # read in the excel file dropping the top junk rows
    ev_ranges_file = "Data/10963_EV_range_efficiency_1-6-22.xlsx"
    ev_ranges_df = pd.read_excel(ev_ranges_file, skiprows=2)

    # Removing unused data columns

    ev_ranges_df.drop(ev_ranges_df.columns[[0,5,6,7,8,9]], axis=1, inplace=True)


    # Removes the trailing NaNs
    ev_ranges_df.dropna(inplace=True)

    # Group by year with mean of ranges then return to a dataframe format
    ev_gbymean = ev_ranges_df.groupby("Model Year").mean().reset_index()

    # Group by year with max of ranges then return to a dataframe format
    ev_gbymax = ev_ranges_df.groupby("Model Year")["Range"].max().reset_index()

    # Merge the two grouped dataframes back to one
    ev_ranges_by_year = pd.merge(ev_gbymean,ev_gbymax, how="left", on="Model Year")

    # Rename the columns
    ev_ranges_by_year.rename(columns={'Range_x' : 'Average Range', 'Range_y' : 'Max Range'}, inplace=True)

    # Remove first two rows for continuous years
    ev_ranges_by_year.drop([0,1], inplace=True)

    ev_ranges_by_year.reset_index(drop=True, inplace=True)
    
    ev_ranges_by_year.reset_index(drop=True, inplace=True)

    # Return final dataframe as a json
    return ev_ranges_by_year.to_dict('split')

In [5]:
# Create an instance of Flask
app = Flask(__name__)

In [6]:
# Use PyMongo to establish Mongo connection
conn = "mongodb://localhost:27017/ev_data"
mongo = PyMongo(app, uri = conn)

In [33]:
ev_range = ranges()

In [34]:
ev_range

{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'columns': ['Model Year', 'Average Range', 'Max Range'],
 'data': [[2010, 200.0, 200.0],
  [2011, 105.4, 245.0],
  [2012, 105.16666666666667, 265.0],
  [2013, 102.16666666666667, 204.0],
  [2014, 96.8, 238.0],
  [2015, 99.4, 253.0],
  [2016, 109.66666666666667, 257.0],
  [2017, 140.73333333333332, 310.0],
  [2018, 159.30769230769232, 298.0],
  [2019, 193.53333333333333, 319.0],
  [2020, 235.06666666666666, 347.0],
  [2021, 255.53846153846155, 361.0]]}

In [35]:
type(ev_range)

dict

In [36]:
mongo.db.ranges.update_one({}, {"$set": ev_range}, upsert=True)

In [37]:
result = mongo.db.ranges.find()

In [38]:
result_list = list(result)

In [39]:
result

In [42]:
if len(result_list) > 0:
        # Return the first result only and strip off the '_id'
        data = result_list[0] 
        id_to_discard = data.pop('_id', None)
else:
        # Construct an error message
        data = {'Error': 'No data found'} 

In [43]:
data

{'columns': ['Model Year', 'Average Range', 'Max Range'],
 'data': [[2010, 200.0, 200.0],
  [2011, 105.4, 245.0],
  [2012, 105.16666666666667, 265.0],
  [2013, 102.16666666666667, 204.0],
  [2014, 96.8, 238.0],
  [2015, 99.4, 253.0],
  [2016, 109.66666666666667, 257.0],
  [2017, 140.73333333333332, 310.0],
  [2018, 159.30769230769232, 298.0],
  [2019, 193.53333333333333, 319.0],
  [2020, 235.06666666666666, 347.0],
  [2021, 255.53846153846155, 361.0]],
 'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]}